<summary>
    <font size="5" color="gray"><b> CONVENIO CENACE-UNISON 2023-2025
 </b></font>
</summary><br>
<summary>
    <font size="4" color="orange"><b> DESARROLLO DE PROTOTIPO: METODOLOGÍA CON APRENDIZAJE PROFUNDO PARA EL PRONÓSTICO A CORTO PLAZO DE DEMANDA DE ENERGÍA CON DATOS EN ADELANTO.
 </b></font>
</summary>

---

<summary>
    <font size="6" color="gray"><b> Libreta JNB1: Ingeniería de características 
 </b></font>
</summary>
<summary>
    <font size="5" color="gray"><b> Modelado parte 1
 </b></font>
</summary>

--- 

<summary>
    <font size="4" color="lightblue"><b> MODELO ENCODER-DECODER CON SEÑALES DE DÍAS FESTIVOS, CLIMA Y DIA EN ADELANTO
 </b></font>
</summary>

---




OLIVIA CAROLINA GUTÚ OCAMPO

REYNA YANET HERNÁNDEZ MADA

JULIO WAISSMAN VILANOVA (Responsable del proyecto)


---

<summary>
    <font size="4" color="orange"><b> INPUTS
 </b></font>
</summary>

Archivo  **'/../MODELO_M3_DATASET_1/internal_inputs/GCRNO_complete_silver_[fecha_de_creación].csv'** creado en la libreta JNB0. 


--- 
<summary>
    <font size="4" color="orange"><b> CHECK POINTS
 </b></font>
</summary>


No hay check points en esta libreta
    

---
<summary>
    <font size="4" color="orange"><b> OUTPUT (INTERNAL INPUTS)
 </b></font>
</summary>

<summary>
    <font size="3" color="gray"><b> 'PCA' folder
 </b></font>
</summary>
   
   * **'./internal_inputs/PCA/dataset_complete_components_train.csv'** : datos de entrenamiento con las primeros 2 componentes PCA agregados.
   * **'./internal_inputs/PCA/scaler_model_clima.pkl'** parámetros del escalamiento estándar aplicado a las variables de clima previo al procedimiento de pca al conjunto de entrenamiento.
   * **'./internal_inputs/PCA/pca_clima.pkl'** : parámetros del procedimiento de pca a las variables de clima del conjunto de entrenamiento.
   * **'./internal_inputs/PCA/dataset_complete_components_[test/train/val] .csv'** : datos de prueba/entrenamiento/validación con las dos primeros 2 componentes PCA agregados, resultado de aplicar los parámetros correspondientes al conjunto de entrenamiento.
    
<summary>
    <font size="3" color="gray"><b> 'MINMAX' folder
 </b></font>
</summary>
    
   * **'./internal_inputs/MINMAX/scalers.pkl'** : parámetros del escalamiento al intervalo $[-1,1]$ aplicado al conjunto de entrenamiento
     
<summary>
    <font size="3" color="gray"><b> 'TENSORS' folder
 </b></font>
</summary>
 
 Tensores de entrada-salida correspondientes al conjunto de entrenamiento, validación y prueba (entradas de la libreta YOO-JNB2)
   
   * **'./internal_inputs/TENSORS/X_encoder_train.npy'** 
   * **'./internal_inputs/TENSORS/X_decoder_train.npy'** 
   * **'./internal_inputs/TENSORS/y_train.npy'** 
   * **'./internal_inputs/TENSORS/X_encoder_val.npy'** 
   * **'./internal_inputs/TENSORS/X_decoder_val.npy'** 
   * **'./internal_inputs/TENSORS/y_val.npy'** 
   * **'./internal_inputs/TENSORS/X_encoder_test.npy'** 
   * **'./internal_inputs/TENSORS/X_decoder_test.npy'** 
   * **'./internal_inputs/TENSORS/y_test.npy'** 

---
<summary>
    <font size="4" color="orange"><b> OUTPUT 
 </b></font>
</summary>

   * **'outputs/model_info.txt'** : archivo con la información relevante sobre el proceso de entrenamiento.

## 1.   Importing libraries and functions

In [1]:
#importar librerías
import numpy as np
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
import os

# Normalización librarías
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
import pickle as pk

# librarías auxiliares
from datetime import datetime, timedelta
from datetime import date

In [2]:
# Importing PCA functions for futher processing
%run 'funciones/Y00pcafunction.ipynb'

## 2.   Acceso a  datos silver

In [3]:
#leer el archivo generado en la libreta Y00
today = date.today().strftime("%Y-%m-%d")

#Guardar nuevo dataframe con  date index
path = os.path.dirname(os.getcwd()) #leer ubicacion carpeta actual

df_GCRNO_complete = pd.read_csv(path + '/internal_inputs/' + 'GCRNO_complete_silver_' + today + '.csv', parse_dates=True)
df_GCRNO_complete['Date_time'] =  pd.to_datetime(df_GCRNO_complete['Date_time'],format='%Y-%m-%d %H:%M:%S')
df_GCRNO_complete.set_index("Date_time", inplace=True)
df_GCRNO_complete = df_GCRNO_complete.asfreq('h')

# Duplicar columna Date-Timed para flexibilidad
df_GCRNO_complete['Date_timed'] = df_GCRNO_complete.index
df_GCRNO_complete.tail(2)

,Hour,Energy_Demand,Day,Month,Tmax-Cab,Tmax-Hmo,Tmax-Obr,Tmax-Lmo,Tmax-Cul,Tmin-Cab,...,Tuesday_Aft_Hol,Easter_week,May_1s,May_10t,Sept_16,Nov_2nd,Before_Christmas_NY,Christmas_NY,After_Christmas_NY,Date_timed
Date_time,,,,,,,,,,,,,,,,,,,,,
2022-05-12 22:00:00,22,3777,3,5,34.0,37.0,38.0,36.0,40.0,15.0,...,0,0,0,0,0,0,0,0,0,2022-05-12 22:00:00
2022-05-12 23:00:00,23,3796,3,5,34.0,37.0,38.0,36.0,40.0,15.0,...,0,0,0,0,0,0,0,0,0,2022-05-12 23:00:00


## 3.   Creación de conjuntos de entrenamiento, validación y prueba

In [4]:
# Definir las fechas de división para los conjuntos de validación y prueba
# **Testing:** Conjunto de Prueba toma en cuenta los ultimos 12 meses
# **Validation** Validación toma en cuenta  4 meses previos al período de prueba
# **Training:** Conjunto de entrenamiento, toma en cuenta el resto de los datos después de dividir  prueba y validación
testing_start = df_GCRNO_complete.index.max() - relativedelta(months = 1) ## PARA LA VERSION FINAL CAMBIAR 1 POR 12
validation_start = testing_start - relativedelta(months = 1)  ## PARA LA VERSION FINAL CAMBIAR 1 POR 4

In [5]:
# Dividir los datos entrenamiento, validación y prueba
training_data = df_GCRNO_complete.loc[df_GCRNO_complete.index <= validation_start]
validation_data = df_GCRNO_complete.loc[(df_GCRNO_complete.index > validation_start) & (df_GCRNO_complete.index <= testing_start)]
testing_data = df_GCRNO_complete.loc[df_GCRNO_complete.index >testing_start]

In [6]:
#Exportar información de set de entrenamiento, validación y prueba al archivo model_info.txt

In [7]:
%%capture cap --no-stderr
print(f'Training data size: {len(training_data)}')
print(f'Validation data size: {len(validation_data)}')
print(f'Testing data size: {len(testing_data)}')

In [8]:
with open('./outputs/model_info.txt', 'w') as f:
    f.write(cap.stdout)

In [9]:
%%capture cap --no-stderr
print('Training set:', training_data.index.min(),'-TO-', training_data.index.max())
print('Validation set:', validation_data.index.min(),'-TO-', validation_data.index.max())
print('Testing set:', testing_data.index.min(),'-TO-',testing_data.index.max())

In [10]:
with open('./outputs/model_info.txt', 'a') as f:
    f.write(cap.stdout)

## 4.    PCA 

In [11]:
# Nuevas columnas en set de entrenamiento
## Se aplica análisis de componentes principales a variables climáticas, después de la estandarización de las mismas
## Se guardan los parámetros de  estandarización y PCA n
## Se ejecuta función 'exogen_pca' :
### 'dataset_complete_components_train' : datos de entrenamiento con primeras 2 componentes de PCA añadidas
### 'pca_clima' :parámetros de entrenamiento de procedimiento PCA
### 'scaler_model_clima' : parámetros de entrenamiento de std scaler
dataset_complete_components_train, pca_clima, scaler_model_clima = exogen_pca(training_data, 'Date_timed')
dataset_complete_components_train.to_csv('./internal_inputs/PCA/dataset_complete_components_train.csv')

## Exportando parámetros de procesamiento para su uso fuera de la libreta
pk.dump(pca_clima, open('./internal_inputs/PCA/pca_clima.pkl','wb'))
pk.dump(scaler_model_clima, open('./internal_inputs/PCA/scaler_model_clima.pkl','wb'))

# Aplicar parámetros de training a
## set de validación
dataset_complete_components_val = transform_data(validation_data,'Date_timed',pca_clima,scaler_model_clima)
dataset_complete_components_val.to_csv('./internal_inputs/PCA/dataset_complete_components_val.csv')
## Test set  
dataset_complete_components_test = transform_data(testing_data,'Date_timed',pca_clima,scaler_model_clima)
dataset_complete_components_test.to_csv('./internal_inputs/PCA/dataset_complete_components_test.csv')

## 5.    MINMAX [-1,1]

In [12]:
Features = ['Energy_Demand', 'Day', 'Hour', 'Month','PC1_Weather','PC2_Weather',
            'Monday_Holiday', 'Tuesday_Aft_Hol', 'Easter_week', 'May_1s', 'May_10t', 
            'Sept_16', 'Nov_2nd', 'Before_Christmas_NY', 'Christmas_NY', 'After_Christmas_NY']

In [13]:
# Reescalar set de entrenamiento
train = dataset_complete_components_train
scalers = {} #dicccionario

for attr in Features:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    s_s = scaler.fit_transform(train[attr].values.reshape(-1,1))
    scalers[attr] = scaler
    train[attr] = s_s.ravel()

#   Exportar parámetros
pk.dump(scalers, open('./internal_inputs/MINMAX/scalers.pkl','wb'))    
    
# Applicando parámetros de training a
## set de validación
val = dataset_complete_components_val
for attr in Features:
    scaler = scalers[attr]
    s_s = scaler.transform(val[attr].values.reshape(-1,1))
    val[attr] = s_s.ravel()
## Test set
test = dataset_complete_components_test
for attr in Features:
    scaler = scalers[attr]
    s_s = scaler.transform(test[attr].values.reshape(-1,1))
    test[attr] = s_s.ravel()

## 6. Generación de tensores

In [14]:
#Ajustando hiperparámetros
n_pasado = (24 * 7) + 12
n_futuro = 36

In [15]:
def divide_series(series, n_pasado, n_futuro):
    """
    n_pasado: number of past observations for the encoder
    n_futuro: number of future observations and inputs given in advance for the decoder
    """
    X_encoder, X_decoder, y = list(), list(), list()  
    generador = range(len(series))

    for ini in generador:
        fin_anterior = ini + n_pasado
        fin_actual = fin_anterior + n_futuro
        if fin_actual > len(series):
            break
        past_encoder = series[ini: fin_anterior, :]
        future_decoder = series[fin_anterior: fin_actual, [1,4,5,6,7,8,9,10,11,12,13,14,15]]
        future_output = series[fin_anterior: fin_actual, :1]
        X_encoder.append(past_encoder)
        X_decoder.append(future_decoder)
        y.append(future_output)
             
    return np.array(X_encoder), np.array(X_decoder), np.array(y)

In [16]:
all_features = ['Energy_Demand', 
                'Day', 
                'Hour', 
                'Month', 
                'PC1_Weather',
                'PC2_Weather',
                'Monday_Holiday', 
                'Tuesday_Aft_Hol', 
                'Easter_week',
                'May_1s', 
                'May_10t', 
                'Sept_16', 
                'Nov_2nd', 
                'Before_Christmas_NY',
                'Christmas_NY', 
                'After_Christmas_NY'
               ]

In [17]:
#Definir tensores
X_encoder_train, X_decoder_train, y_train = divide_series(train[all_features].values, n_pasado, n_futuro)
X_encoder_val, X_decoder_val, y_val = divide_series(val[all_features].values, n_pasado, n_futuro)
X_encoder_test, X_decoder_test, y_test = divide_series(test[all_features].values, n_pasado, n_futuro) 

## 7. Guardado de tensores

In [18]:
%%capture cap --no-stderr
print('X_encoder_train tensor size:', X_encoder_train.shape,'\n', 
      'X_decoder_train tensor size:', X_decoder_train.shape,'\n',
      'y_train tensor size:', y_train.shape
     )
print('X_encoder_val tensor size:', X_encoder_val.shape,'\n', 
      'X_decoder_val tensor size:', X_decoder_val.shape,'\n',
      'y_val tensor size:', y_val.shape
     )
print('X_encoder_test tensor size:', X_encoder_test.shape,'\n', 
      'X_decoder_test tensor size:', X_decoder_test.shape,'\n',
      'y_test tensor size:', y_test.shape
     )
print('DATE:', today,'\n'
     )

In [19]:
with open('./outputs/model_info.txt', 'a') as f:
    f.write(cap.stdout)

In [20]:
np.save('./internal_inputs/TENSORS/X_encoder_train.npy', X_encoder_train)
np.save('./internal_inputs/TENSORS/X_decoder_train.npy', X_decoder_train)
np.save('./internal_inputs/TENSORS/y_train.npy', y_train)
np.save('./internal_inputs/TENSORS/X_encoder_val.npy', X_encoder_val)
np.save('./internal_inputs/TENSORS/X_decoder_val.npy', X_decoder_val)
np.save('./internal_inputs/TENSORS/y_val.npy', y_val)
np.save('./internal_inputs/TENSORS/X_encoder_test.npy', X_encoder_test)
np.save('./internal_inputs/TENSORS/X_decoder_test.npy', X_decoder_test)
np.save('./internal_inputs/TENSORS/y_test.npy', y_test)

<summary>
    <font size="4" color="gray"> Maestría en Ciencia de Datos | Universidad de Sonora </font>
</summary>
<font size="1" color="gray"> Blvd. Luis Encinas y Rosales s/n Col. Centro. Edificio 3K1 planta baja C.P. 83000, Hermosillo, Sonora, México </font>
<font size="1" color="gray"> mcd@unison.mx </font>
<font size="1" color="gray"> Tel: +52 (662) 259 2155  </font>